In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import h5py
import os
import re

In [2]:
def getproj(z):
    #global rscale, thetascale, phiscale, theta_index
    theta_inverse = thetascale[int(temp.shape[2]/2):][::-1]
    theta_index = np.searchsorted((z / np.cos(theta_inverse)), r_outer)
    r_desired = (z / np.cos(theta_inverse))[:theta_index]
    r_projected = np.array([a*b for a,b in zip(r_desired, np.sin(theta_inverse[:theta_index]))])
    phi_pad = np.append(phiscale[:], 2*np.pi)
    r_proj_pad   = np.pad(r_projected[:], ((0,1)), mode='constant', constant_values=(0,r_projected[-1]))
    s_mesh, phi_mesh = np.meshgrid(r_proj_pad,phi_pad)
    return r_desired, theta_index, s_mesh, phi_mesh

In [3]:
def getplotdata(r_desired):
    #global inter_index, inter_temp
    inter_index = np.array([np.searchsorted(rscale,r_desired[i]) for i in range(theta_index)])
    inter_index = np.array([inter_index - 1, inter_index]).T
    inter_temp = temp[:,:,int(temp.shape[2]/2):,:]
    inter_temp = np.flip(inter_temp,2)
    plot_data = [[np.interp(r_desired[i], rscale[inter_index[i]], j) for j in inter_temp[:,:,i,inter_index[i]].reshape(-1,2)] for i in range(theta_index)]
    plot_data = np.array(plot_data).T.reshape(temp.shape[0],-1,theta_index)
    return plot_data

In [4]:
files = []
mydir = "checkpoint_test"
for file in os.listdir(mydir):
    if file.endswith(".h5"):
        files.append(os.path.join(mydir, file))
files.sort(key=lambda f: int(re.sub('\D','', f)))

r_inner = 7/13.
r_outer = 20./13.

In [5]:
counter = 0
for file in files:
    df = h5py.File(file,"r")
    temp = df['tasks']['T']
    
    if counter == 0:
        rscale = temp.dims[3][0][:]
        thetascale = temp.dims[2][0][:]
        phiscale = temp.dims[1][0][:]
        r_desired, theta_index, s_mesh, phi_mesh = getproj(0.5*(r_outer-r_inner)+r_inner)
    
    temp = np.array(temp)
    
    plot_data = getplotdata(r_desired)
    
    for data in plot_data:
        fig, ax = plt.subplots(subplot_kw=dict(polar=True))
        pcm = ax.pcolormesh(phi_mesh, s_mesh, data, cmap='RdYlBu_r')
        plt.colorbar(pcm)
        plt.savefig(str(mydir)+'/%s_%04i.png' %("plot".format(0), counter), dpi=150)
        plt.close(fig)
        counter += 1